# Command Pattern
"The Command Pattern encapsulates a request as an object, thereby letting you parameterize other objects with different requests, queue or log requests, and support undoable operations."

In [20]:
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass
    @abstractmethod
    def undo(self):
        pass


class Light:
    def __init__(self, name : str):
        self.name = name

    def on(self):
        print(f"Turn {self.name} light on")
    def off(self):
        print(f"Turn {self.name} light off")

# Command
class LightOnCommand(Command):
    def __init__(self, light : Light):
        self.light = light

    def execute(self):
        self.light.on()

    def undo(self):
        self.light.off()

# Invoker
class SimpleRemoteControl:

    def __init__(self):
        self.slot : Command = None

    def setCommand(self, command : Command):
        self.slot = command

    def buttonWasPresed(self):
        self.slot.execute()

# Testing

In [21]:
simple_remote = SimpleRemoteControl()
light = Light("Living_room")
light_on_command = LightOnCommand(light)

simple_remote.setCommand(light_on_command)
simple_remote.buttonWasPresed()


Turn Living_room light on


In [30]:
from typing import List

class NoCommand(Command):
    def execute(self):
        pass

    def undo(self):
        pass

class LightOffCommand(Command):
    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        self.light.off()

    def undo(self):
        self.light.on()

class LightOnCommand(Command):
    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        self.light.on()

    def undo(self):
        self.light.off()


class Stereo:
    def __init__(self, name : str):
        self.name = name
    def on(self):
        print(f"Stereo {self.name} on")
    def setCD(self):
        print(f"Stereo {self.name} cd set")
    def setVolume(self, volume_level: int):
        print(f"Stereo {self.name} Volumne level set to {volume_level}")
    def off(self):
        print(f"Stereo {self.name} off")

class StereoOnWithCDCommand(Command):
    def __init__(self, stereo : Stereo):
        self.stereo = stereo

    def execute(self):
        self.stereo.on()
        self.stereo.setCD()
        self.stereo.setVolume(11)

    def undo(self):
        self.stereo.off()

class StereoOffCommand(Command):
    def __init__(self, stereo : Stereo):
        self.stereo = stereo

    def execute(self):
        self.stereo.off()

    def undo(self):
        self.stereo.on()
        self.stereo.setCD()
        self.stereo.setVolume(11)


class Macrocommand(Command):
    def __init__(self, commands : List[Command]):
        self.commands = commands

    def execute(self):
        for command in self.commands:
            command.execute()

    def undo(self):
        for command in self.commands:
            command.undo()


class RemoteControl:

    NUM_SLOTS = 7

    def __init__(self):
        self.off_commands : List[Command] = []
        self.on_commands : List[Command] = []
        self.undo_command : Command = NoCommand()

        for x in range(0,self.NUM_SLOTS):
            self.on_commands.append(NoCommand())
            self.off_commands.append(NoCommand())


    def set_command(self, slot : int, command_on : Command, command_off : Command):
        self.on_commands[slot] = command_on
        self.off_commands[slot] = command_off

    def on_button_was_pushed(self, slot: int):
        self.on_commands[slot].execute()
        self.undo_command = self.on_commands[slot]

    def off_button_was_pushed(self, slot : int):
        self.off_commands[slot].execute()
        self.undo_command = self.off_commands[slot]

    def undo_button_was_pressed(self):
        self.undo_command.undo()

    def __str__(self):
        output = ""

        for x in range(0, self.NUM_SLOTS):
            output += f"[slot {x}] {self.on_commands.__str__()}    { self.off_commands.__str__()}\n"
        return output



# Testing

In [32]:
remote_control = RemoteControl()

living_room_light = Light("Living Room")
kitchen_light = Light("Kitchen")
living_room_light_on = LightOnCommand(living_room_light)
living_room_light_off = LightOffCommand(living_room_light)
kitchen_light_on = LightOnCommand(kitchen_light)
kitchen_light_off = LightOffCommand(kitchen_light)

stereo = Stereo("Living Room")
stereo_on_with_cd_command = StereoOnWithCDCommand(stereo)
stereo_off_command = StereoOffCommand(stereo)

party_on = [living_room_light_on, stereo_on_with_cd_command, kitchen_light_off]
party_off = [living_room_light_off, stereo_off_command, kitchen_light_on]
party_on_command = Macrocommand(party_on)
party_off_command = Macrocommand(party_off)

remote_control.set_command(0, living_room_light_on, living_room_light_off)
remote_control.set_command(1, kitchen_light_on, kitchen_light_off)
remote_control.set_command(2, stereo_on_with_cd_command, stereo_off_command)
remote_control.set_command(3, party_on_command, party_off_command)

print(remote_control)

remote_control.on_button_was_pushed(0)
remote_control.off_button_was_pushed(0)
remote_control.on_button_was_pushed(1)
remote_control.off_button_was_pushed(1)
remote_control.on_button_was_pushed(2)
remote_control.off_button_was_pushed(2)
remote_control.undo_button_was_pressed()
print("Party button pressed")
remote_control.on_button_was_pushed(3)
print("Party turned off")
remote_control.off_button_was_pushed(3)

[slot 0] [<__main__.LightOnCommand object at 0x10d2c7950>, <__main__.LightOnCommand object at 0x10d2b4a50>, <__main__.StereoOnWithCDCommand object at 0x10d2c4ed0>, <__main__.Macrocommand object at 0x10d2c4290>, <__main__.NoCommand object at 0x10d20f2d0>, <__main__.NoCommand object at 0x10d20f510>, <__main__.NoCommand object at 0x10d20c190>]    [<__main__.LightOffCommand object at 0x10d2c7ed0>, <__main__.LightOffCommand object at 0x10d2c68d0>, <__main__.StereoOffCommand object at 0x10d2c4350>, <__main__.Macrocommand object at 0x10d2e5990>, <__main__.NoCommand object at 0x10d20fad0>, <__main__.NoCommand object at 0x10d20c150>, <__main__.NoCommand object at 0x10d20f0d0>]
[slot 1] [<__main__.LightOnCommand object at 0x10d2c7950>, <__main__.LightOnCommand object at 0x10d2b4a50>, <__main__.StereoOnWithCDCommand object at 0x10d2c4ed0>, <__main__.Macrocommand object at 0x10d2c4290>, <__main__.NoCommand object at 0x10d20f2d0>, <__main__.NoCommand object at 0x10d20f510>, <__main__.NoCommand obje